In [1]:
run create_map.py

/home/users/marcyin/miniconda3/lib/python3.7/site-packages/traitlets/traitlets.py:588: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


Label(value='')

In [2]:
my_map

Map(center=[9.700554008484533, -0.541705192919114], controls=(ZoomControl(options=['position', 'zoom_in_text',…